In [140]:
import numpy as np
import pandas as pd
import cv2
import json
from sklearn.metrics.pairwise import cosine_similarity

In [168]:
# load image
image = cv2.imread('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/screenshots/Board6.png')

# get ref grid
f = open('hex_positions.json')
hex_centers = json.load(f)

# load token vectors
token_avgs = pd.read_csv('average_token_vectors.csv', index_col=0)

# load token color ranges
f = open('token_color_ranges.json')
color_ranges = json.load(f)

# load animal patterns
f = open('animal_patterns.json')
animals = json.load(f)

In [34]:
# convert to gray and blur
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.blur(image, (10,10))

# detect edges
edges = cv2.Canny(gray, 50, 150)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# crop to game board
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)
board_region = image[y:y+h, x:x+w] 

In [124]:
cv2.imshow('', image)
cv2.waitKey(0)

-1

In [36]:
cv2.imwrite('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/cropped_Board6.jpg', board_region)

True

In [150]:
# Define token and cube colors (RGB)
color_ranges = {
    "red": ((158, 37, 45), (253, 131, 122)),  
    "blue": ((29, 118, 127), (126, 223, 236)),
    "green": ((126, 130, 23), (211, 222, 103)),
    "yellow": ((202, 167, 25), (255, 231, 103)),
    "gray": ((109, 109, 109), (183, 180, 178)),  
    "brown": ((132, 82, 47), (147, 95, 59)),
    "board":((223, 202, 168), (234, 214, 185)),
    "spirit": ((159, 127, 88), (252, 248, 242)),
    "animal": ((198, 63, 28), (255, 156, 92))
}

In [136]:
# token & cube detection
rgb = cv2.cvtColor(board_region, cv2.COLOR_BGR2RGB)

# region of interest (ROI) parameters
roi_w = 10
roi_l = 25

# Extract ROI for each hex
for (hex_x, hex_y) in hex_centers:
    roi = rgb[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]  # Adjust based on hex size

    # Apply color masks to count token pixels
    for token_type, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(roi, np.array(lower), np.array(upper))
        pixel_count = cv2.countNonZero(mask)

        if pixel_count > 50:  # Threshold to ignore small noise
            detected_tokens.append(token_type)

    # Store detected tokens
    board_state["tokens"].append({
        "x": hex_x, "y": hex_y,
        "stack": len(detected_tokens),
        "types": detected_tokens
    })

In [138]:
roi_w = 10
roi_l = 25

for (hex_x, hex_y) in hex_centers:
    top_left = (hex_x - roi_w, hex_y - roi_l)
    bottom_right = (hex_x + roi_w, hex_y + roi_l)
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)  # Green rectangle for debugging


cv2.imshow('', image)
cv2.waitKey(0)

-1

In [137]:
print(board_state['tokens'][0])

{'x': 27, 'y': 25, 'stack': 1, 'types': ['gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'yellow', 'gray', 'red', 'gray', 'red', 'gray', 'blue', 'gray', 'red', 'gray', 'red', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'red', 'blue', 'gray', 'red', 'blue', 'gray', 'blue', 'gray', 'green', 'yellow', 'brown', 'red', 'gray', 'red', 'gray', 'gray', 'brown', 'yellow', 'gray', 'red', 'gray

In [ ]:
def detect_top_colors(roi, step=10):
    """
    Detects the primary colors in an ROI, scanning from top to bottom in horizontal slices.
    - step: The vertical pixel step size to reduce redundant detections.
    """
    detected_tokens = []
    roi_height = roi.shape[0]
    
    for y in range(0, roi_height, step):  # Scan in steps to avoid repeated detections
        row = roi[y:y+step, :]  # Extract horizontal slice
        for token_type, (lower, upper) in color_ranges.items():
            mask = cv2.inRange(row, np.array(lower), np.array(upper))
            if cv2.countNonZero(mask) > STACK_THRESHOLD:
                if token_type not in detected_tokens:
                    detected_tokens.append(token_type)  # Keep order of detection
                    break  # Stop once a color is found in this row

    return detected_tokens

def detect_stack_levels(roi):
    """
    Determines how many stacked tokens are in the hex by analyzing vertical color changes.
    """
    roi_height = roi.shape[0]
    detected_stacks = []
    prev_color = None

    for y in range(roi_height):
        row = roi[y:y+1, :]  # Single-pixel row scan
        for token_type, (lower, upper) in color_ranges.items():
            mask = cv2.inRange(row, np.array(lower), np.array(upper))
            if cv2.countNonZero(mask) > STACK_THRESHOLD:
                if token_type != prev_color:  # Register color change
                    detected_stacks.append(token_type)
                    prev_color = token_type
                break  # Stop checking after first detected color in this row

    return len(detected_stacks), detected_stacks  # Number of detected stack levels


def detect_cube(roi):
    """
    Checks the top part of the ROI to detect if a cube is placed on top.
    """
    cube_roi = roi[0:int(roi.shape[0] * 0.2), :]  # Top 20% of ROI
    cube_mask = cv2.inRange(cube_roi, np.array(color_ranges["cube"][0]), np.array(color_ranges["cube"][1]))
    
    return cv2.countNonZero(cube_mask) > CUBE_THRESHOLD  # Returns True if cube detected


In [ ]:
roi_w = 10
roi_l = 25

for (hex_x, hex_y) in hex_centers:
    roi = hsv[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]  # Extract region

    # Get stack details
    stack_count, detected_tokens = detect_stack_levels(roi)

    # Get cube presence
    has_cube = detect_cube(roi)

    board_state["tokens"].append({
        "x": hex_x, "y": hex_y,
        "stack": stack_count,
        "types": detected_tokens,
        "cube": has_cube
    })


In [151]:
for (hex_x, hex_y) in hex_centers:
    roi = rgb[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]

    print(f"\nHex at ({hex_x}, {hex_y})")

    for token_type, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(roi, np.array(lower), np.array(upper))
        pixel_count = cv2.countNonZero(mask)

        # Special case for spirit color: subtract board pixels
        if token_type == "spirit":
            board_mask = cv2.inRange(roi, np.array(color_ranges["board"][0]), np.array(color_ranges["board"][1]))
            spirit_adjusted_count = max(0, pixel_count - cv2.countNonZero(board_mask))  # Ensure non-negative

            print(f"  spirit (raw): {pixel_count} pixels")
            print(f"  spirit (adjusted): {spirit_adjusted_count} pixels (spirit - board)")
        else:
            print(f"  {token_type}: {pixel_count} pixels")



Hex at (27, 25)
  red: 0 pixels
  blue: 0 pixels
  green: 0 pixels
  yellow: 0 pixels
  gray: 430 pixels
  brown: 0 pixels
  board: 118 pixels
  spirit (raw): 309 pixels
  spirit (adjusted): 191 pixels (spirit - board)
  animal: 0 pixels

Hex at (27, 75)
  red: 80 pixels
  blue: 0 pixels
  green: 3 pixels
  yellow: 12 pixels
  gray: 212 pixels
  brown: 2 pixels
  board: 13 pixels
  spirit (raw): 71 pixels
  spirit (adjusted): 58 pixels (spirit - board)
  animal: 128 pixels

Hex at (27, 125)
  red: 0 pixels
  blue: 0 pixels
  green: 10 pixels
  yellow: 332 pixels
  gray: 72 pixels
  brown: 0 pixels
  board: 264 pixels
  spirit (raw): 461 pixels
  spirit (adjusted): 197 pixels (spirit - board)
  animal: 4 pixels

Hex at (27, 175)
  red: 76 pixels
  blue: 0 pixels
  green: 8 pixels
  yellow: 8 pixels
  gray: 221 pixels
  brown: 0 pixels
  board: 279 pixels
  spirit (raw): 419 pixels
  spirit (adjusted): 140 pixels (spirit - board)
  animal: 124 pixels

Hex at (27, 225)
  red: 79 pixels
 

In [108]:
def classify_vector(input_vector, token_avgs):
    """
    Given a pixel count vector, return the top matching label from averaged vectors.
    
    Args:
        input_vector (dict): e.g., {'red': 0, 'blue': 172, ..., 'animal': 133}
        token_vectors (pd.DataFrame) - database of tokens vectors, averaged from labeled data

    Returns:
        list of tuples: [(label, similarity_score), ...]
    """
    
    # Ensure consistent order
    token_order = token_avgs.index.tolist()
    input_array = np.array([[input_vector.get(token, 0) for token in token_order]])  # shape (1, n_tokens)

    # Transpose for cosine similarity: now shape (n_labels, n_tokens)
    label_vectors = token_avgs.T.values
    label_names = token_avgs.columns.tolist()

    # Compute cosine similarity
    similarities = cosine_similarity(input_array, label_vectors)[0]
    best_index = similarities.argmax()

    return label_names[best_index]


In [143]:
test_vector = {
    "red": 1,
    "blue": 0,
    "green": 7,
    "yellow": 336,
    "gray": 49,
    "brown": 20,
    "board": 290,
    "spirit": 207,
    "animal": 4
}


token_order = token_avgs.index.tolist()
input_array = np.array([[test_vector.get(token, 0) for token in token_order]])  # shape (1, n_tokens)

label_vectors = token_avgs.T.values
label_names = token_avgs.columns.tolist()

similarities = cosine_similarity(input_array, label_vectors)[0]
best_index = similarities.argmax()

In [105]:
def extract_hex_vectors(image, hex_centers, roi_w, roi_l, color_ranges):
    """
    Extracts color pixel count vectors from a hex grid in the image.

    Args:
        image: BGR image (as read by cv2.imread)
        hex_centers: List of (x, y) tuples representing hex centers
        roi_w, roi_l: Half-width and half-height of the ROI box around each center
        color_ranges: Dict of token_type -> (lower_bgr, upper_bgr)

    Returns:
        List of dicts: [{'coord': [x, y], 'vector': {'red': ..., 'blue': ..., ...}}, ...]
    """
    vectors = []
    
    # convert to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for (hex_x, hex_y) in hex_centers:
        roi = image[hex_y - roi_l : hex_y + roi_l, hex_x - roi_w : hex_x + roi_w]
        vector = {}

        for token_type, (lower, upper) in color_ranges.items():
            lower_np = np.array(lower, dtype=np.uint8)
            upper_np = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(roi, lower_np, upper_np)
            pixel_count = cv2.countNonZero(mask)

            # Adjust spirit by subtracting board pixels
            if token_type == "spirit":
                board_mask = cv2.inRange(roi, np.array(color_ranges["board"][0]), np.array(color_ranges["board"][1]))
                pixel_count = max(0, pixel_count - cv2.countNonZero(board_mask))

            vector[token_type] = int(pixel_count)

        vectors.append({
            "coord": [hex_x, hex_y],
            "vector": vector
        })

    return vectors

In [169]:
# set up LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama  
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA

# load vector DB and embedder
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory="vector_db", embedding_function=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# wake up Mistral
llm = Ollama(model="mistral") 

# load test board
f = open('test_Board6_hex_labeled.json')
labels = json.load(f)

In [193]:
animal_match_prompt = PromptTemplate(
    input_variables=["animals", "board_text"],
    template="""
You are a board game AI assistant tasked with identifying which **spirit animal pattern** appears on a given board.

Each animal pattern includes:
- A central tile (dx=0, dy=0), with relative hex coordinates using a **doubled-height hex grid** system.
- In the **doubled-height hex grid** system, vertical movement changes y-coordinates by 2 and movement to horizontal offset neighbors changes x-coordinates by 1 and y by 1. 
- Optional conditions:
    - `terrain`: token color on that tile, if a single token is required 
    - `stack_condition`: exact order and height of tokens, if multiple tokens are required
    - `cube`: if true, a cube (animal or spirit) must be present on that tile

The board is sparse and may contain empty tiles. 
Each tile contains a list of tokens (top to bottom) or may be empty (`[]`). Example stack: `["animal", "gray"]` means a gray token with an animal cube on top.

Each entry is a hex with:
- `"coord"`: the (x, y) location in doubled-height hex coordinates
- `"label"`: a list of tokens from bottom to top (can be empty)

You must:
1. Check each animal pattern against every possible board hex as a potential center.
2. Consider all valid **rotations and reflections** of each pattern.
3. Match patterns **exactly** — tokens, stack height, cube, and terrain must match.
4. There is **exactly one** spirit on the board.

---

### Spirit Patterns
{animals}

---

### Board State
{board_text}

---

### Task
Identify the **one** spirit pattern that matches the board.

Only output JSON in this format:
{{
  "spirit": "NAME OF MATCHING SPIRIT",
  "pos": "HEX COORDINATES OF MATCHING SPIRIT"
}}
"""
)

chain = LLMChain(
    llm=llm,
    prompt=animal_match_prompt 
)

# Run LLMChain
result = chain.run({
    "animals": animals,
    "labels": labels,
})

print(result)

In [200]:
ascii_prompt = PromptTemplate(
    input_variables=["labels"],
    template="""
You are an expert in board games and spatial reasoning.
You are given a board state from a hexagonal grid using the **doubled-height coordinate system**, where vertical moves change y by 2 and diagonal neighbors differ by ±1 in both x and y.

The board is sparse and may contain empty tiles. 
Each tile contains a list of tokens (top to bottom) or may be empty (`[]`). Example stack: `["animal", "gray"]` means a gray token with an animal cube on top.

Each entry is a hex with:
- `"coord"`: the (x, y) location in doubled-height hex coordinates
- `"label"`: a list of tokens from bottom to top (can be empty)

Your job is to display this board as an ASCII grid so that it's easy for a human to visually confirm the structure.

### Guidelines:
- Include the coordinates of the hexagon labeled in the file.
- Use one letter per color (G=green, R=red, B=blue, Y=yellow, Br=brown, Gr=gray, A=animal, S=spirit)

Here's an example of the output I want:
(0,0): A,G    (2,0): R     (4,0): .
  (1,1): Y     (3,1): B     (5,1): .
(0,2): G     (2,2): .     (4,2): Y

---

### Input Board
{labels}

---

### ASCII Output
Show the board below using ASCII formatting as described.
"""
)


chain = LLMChain(
llm=llm,
prompt=ascii_prompt)

# Run LLMChain
result = chain.run({
    "labels": labels,
})

print(result)
    

 (0,0): Y,A      (0,2): R,S,R    (0,4): B,A      (0,6): Br,Br
  (1,1): Y          (1,3): G       (1,5): A,B,B     (1,7): A,B
(2,0): B           (2,2): Y       (2,4): R,R      (2,6): A,B
  (3,1): R,A        (3,3): G       (3,5): A       (3,7): B
(4,0): B           (4,2): A       (4,4): .      (4,6): A     (4,8): Br

Explanation: I used the given format and abbreviated colors as specified. The top-to-bottom order of tokens is preserved for each hexagon, and empty hexagons are represented with a single period (.). The coordinates match those in the input board.


In [203]:
from openai import OpenAI

with open("key.txt", "r") as f:
    api_key = f.read().strip()
print("API key loaded successfully.")

API key loaded successfully.


In [207]:
# Convert to nicely formatted string
board_text = json.dumps(labels, indent=2)

# Define the prompt
prompt = f"""
You are given a board state from a hexagonal grid using the **doubled-height coordinate system**, where vertical moves change y by 2 and diagonal neighbors differ by ±1 in both x and y.

The board is sparse and may contain empty tiles. 
Each tile contains a list of tokens (top to bottom) or may be empty (`[]`). Example stack: `["animal", "gray"]` means a gray token with an animal cube on top.

Each entry is a hex with:
- `"coord"`: the (x, y) location in doubled-height hex coordinates
- `"label"`: a list of tokens from bottom to top (can be empty)

Your job is to display this board as an ASCII grid so that it's easy for a human to visually confirm the structure.

### Guidelines:
- Include the coordinates of the hexagon labeled in the file.
- Use one letter per color (G=green, R=red, B=blue, Y=yellow, Br=brown, Gr=gray, A=animal, S=spirit)

Here's an example of the output I want:
(0,0): A,G    (2,0): R     (4,0): .
  (1,1): Y     (3,1): B     (5,1): .
(0,2): G     (2,2): .     (4,2): Y

---

### Input Board
{board_text}

---

### ASCII Output
Show the board below using ASCII formatting as described.
"""

client = OpenAI(
  api_key=api_key
)

# Send to OpenAI
response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # or "gpt-3.5-turbo" if needed
    messages=[
        {"role": "system", "content": "You are an expert in board games and spatial reasoning."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2  # Lower = more consistent output
)


In [209]:
# Print the result
print(response.choices[0].message.content)

Based on the input board provided, here is the ASCII representation of the hexagonal grid using the doubled-height coordinate system:

```
(0,0): Y      (2,0): B      (4,0): B
  (1,1): Y      (3,1): A,R,Br   (5,1): .
(0,2): S,R,R  (2,2): Y      (4,2): A,G
  (1,3): G      (3,3): G      (5,3): .
(0,4): A,B    (2,4): R,R    (4,4): .
  (1,5): A,B    (3,5): A,G    (5,5): .
(0,6): Br     (2,6): A,B    (4,6): A,G
  (1,7): A,B    (3,7): B      (5,7): .
(0,8): Br,Br  (2,8): A,R,Gr  (4,8): Br
```

Explanation:
- Each hexagon's coordinates are displayed in parentheses.
- The tokens within each hexagon are represented by their first letter (as per the guidelines provided), with the order from bottom to top.
- A period (`.`) is used to represent an empty hexagon.
- The hexagons are staggered to represent the hexagonal grid visually.
- The `none` label has been interpreted as an empty hexagon.


In [215]:
animal_match_prompt = f"""
Your task is to identify which **spirit animal pattern** appears on a given board state.

Each entry in the given animal pattern JSON includes:
- A central tile (dx=0, dy=0), with other tiles in the pattern defined by relative hex coordinates using a **doubled-height hex grid** system.
- In the **doubled-height hex grid** system, vertical movement changes y-coordinates by 2 and movement to horizontal offset neighbors changes x-coordinates by 1 and y by 1. 
- Optional conditions:
    - `terrain`: token color on that tile, if a single token is required 
    - `stack_condition`: exact order and height of tokens, if multiple tokens are required
    - `cube`: if true, the pattern requires a cube (animal or spirit) on that tile

The board is sparse and may contain empty tiles. 
Each tile contains a list of tokens (top to bottom) or may be empty (`[]`). Example stack: `["animal", "gray"]` means a gray token with an animal cube on top.

Each entry in the given board_text is a hex with:
- `"coord"`: the (x, y) location in doubled-height hex coordinates
- `"label"`: a list of tokens from bottom to top (can be empty)

You must:
1. Check each animal pattern against every possible board hex in board_text as a potential center.
2. Identify where the 'spirit' cube is placed.
3. Match the hex with the spirit cube and it's adjacent hexes with a pattern from the animals JSON.
2. Consider all valid **rotations and reflections** of each pattern.

Considerations:
- the pattern must match exactly: token color, stack order, adjacent tiles, and placement of cube
- there is exactly one pattern match, given that there is one spirit cube

---

### Spirit Patterns
{animals}

---

### Board State
{board_text}

---

### Output

Only output using a JSON in this format:
{{
  "spirit": "NAME OF MATCHING SPIRIT",
  "pos": "HEX COORDINATES OF MATCHING SPIRIT"
}}

"""

client = OpenAI(
  api_key=api_key
)

# Send to OpenAI
response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # or "gpt-3.5-turbo" if needed
    messages=[
        {"role": "system", "content": "You are an expert in board games and spatial reasoning."},
        {"role": "user", "content": animal_match_prompt}
    ],
    temperature=0.2  # Lower = more consistent output
)

In [216]:
print(response.choices[0].message.content)

To solve this task, we need to iterate over each hex on the board, checking for the presence of a spirit cube. Once we find the spirit cube, we'll attempt to match the surrounding hexes with one of the spirit animal patterns, considering all valid rotations and reflections.

First, let's identify the hex that contains the spirit cube:

```json
{
  "hex_coord": [
    0,
    2
  ],
  "label": "spirit, red, red"
}
```

The spirit cube is located at hex coordinates (0, 2). Now we need to match this with one of the spirit animal patterns. We'll check each pattern to see if it fits the board state around the spirit cube's location.

After examining the patterns and the board state, we find that the "stork (spirit)" pattern matches the board state around the spirit cube's location. Here's how the pattern matches:

- The central tile of the pattern is at (0, 2) with a stack condition of height 2, top red, and bottom options that include red, which matches the stack "spirit, red, red".
- The ti

In [224]:
def rotate_doubled(dx, dy, times=1):
    for _ in range(times % 6):
        dx, dy = -dy, dx + dy
    return dx, dy

In [227]:
test_pattern = animals[0]

print(rotate_doubled(test_pattern['pattern'][1]['dx'], test_pattern['pattern'][1]['dy'], times = 1))

(1, -2)


In [221]:
print(test_pattern)

{'animal': 'domestic cat (spirit)', 'pattern': [{'dx': 0, 'dy': 0, 'terrain': 'green'}, {'dx': -1, 'dy': -1, 'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}}, {'dx': 1, 'dy': 1, 'stack_condition': {'height': 2, 'top': 'red', 'bottom_options': ['gray', 'brown', 'red']}, 'cube': True}]}
